# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here
db= pd.read_csv('Admission_Predict.csv')
db

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
db.columns = db.columns.str.strip()
db.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
db.isnull().sum()>0


Serial No.           False
GRE Score            False
TOEFL Score          False
University Rating    False
SOP                  False
LOR                  False
CGPA                 False
Research             False
Chance of Admit      False
dtype: bool

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
db.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


In [7]:
db.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [8]:
# your code here
db_2 = db.copy()
db_2.dtypes

Serial No.             int64
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object

In [9]:
db_2['GRE Score']= db_2['GRE Score'].astype(str)

In [10]:
db_2['CGPA'] = db_2['CGPA'].astype(str)

In [11]:
db_2['GRE_CGPA'] = db_2[['GRE Score', 'CGPA']].agg(' - '.join, axis=1)
db_2['GRE_CGPA']

0      337 - 9.65
1       316 - 8.0
2      322 - 8.67
3      314 - 8.21
4      330 - 9.34
          ...    
380    324 - 9.04
381    325 - 9.11
382    330 - 9.45
383    312 - 8.78
384    333 - 9.66
Name: GRE_CGPA, Length: 385, dtype: object

In [12]:
db_2= db_2.set_index('GRE_CGPA')

In [13]:
cols = ['GRE Score', 'CGPA']
db_2.drop(columns=cols, inplace=True)

In [14]:
db_2.head()

,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE_CGPA,,,,,,,
337 - 9.65,1,118,4,4.5,4.5,1,0.92
316 - 8.0,2,104,3,3.0,3.5,1,0.72
322 - 8.67,3,110,3,3.5,2.5,1,0.80
314 - 8.21,4,103,2,2.0,3.0,0,0.65
330 - 9.34,5,115,5,4.5,3.0,1,0.90


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [15]:
# your code here
db_2.reset_index(inplace=True)

In [16]:
db_2.head()

,GRE_CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337 - 9.65,1,118,4,4.5,4.5,1,0.92
1,316 - 8.0,2,104,3,3.0,3.5,1,0.72
2,322 - 8.67,3,110,3,3.5,2.5,1,0.80
3,314 - 8.21,4,103,2,2.0,3.0,0,0.65
4,330 - 9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [17]:
# your code here
len((db[(db['CGPA']>9)&(db['Research']==1)]).index)

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [18]:
db

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


In [19]:
# your code here
db[(db['CGPA']>9)&(db['SOP']<3.5)]['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [20]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return (col - np.mean(col))/np.std(col)
    
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [21]:
# your code here
db['CGPA_std'] = standardize(db['CGPA'])
db['GRE_std'] = standardize(db['GRE Score'])
db['LOR_std']  = standardize(db['LOR'])

In [22]:
admissions = db
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [23]:
# Libraries
from random import choices

In [24]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])


In [25]:
decision_choice

['LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CG

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [26]:
# your code here
#The lookup() function returns label-based "fancy indexing" function for DataFrame.
#Given equal-length arrays of row and column labels, return an array of the values corresponding to each (row, col) pair.

In [27]:
admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)


/var/folders/h3/qq1rdz_91c9crccsnv0ydvv80000gn/T/ipykernel_61313/796281577.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)


In [28]:
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.063450
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [29]:
# your code here
def decision(row):
    for element in admissions['deciding_column']:
        if element > 0.8:
            return 1
        else:
            return 0
    

In [30]:
admissions.dtypes

Serial No.             int64
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
CGPA_std             float64
GRE_std              float64
LOR_std              float64
deciding_column      float64
dtype: object

In [31]:
admissions['decision']=admissions.apply(decision, axis=1)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [32]:
# your code here
admissions[admissions['decision']==1]['Serial No.'].count()

385

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [33]:
# your code here
admissions.columns = admissions.columns.str.replace(' ', '_')
admissions.columns = admissions.columns.str.lower()
admissions.columns = admissions.columns.str.replace('.', '')

/var/folders/h3/qq1rdz_91c9crccsnv0ydvv80000gn/T/ipykernel_61313/198680302.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  admissions.columns = admissions.columns.str.replace('.', '')


In [34]:
admissions.columns

Index(['serial_no', 'gre_score', 'toefl_score', 'university_rating', 'sop',
       'lor', 'cgpa', 'research', 'chance_of_admit', 'cgpa_std', 'gre_std',
       'lor_std', 'deciding_column', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [39]:
# your code here

def adjusted(row):
        for element in admissions['university_rating']:
            if element >= 4:
                return row+10
            else:
                return row


In [45]:
admissions['adjusted_gre'] =admissions['gre_score'].apply(adjusted)

In [46]:
admissions['adjusted_gre']

0      347
1      326
2      332
3      324
4      340
      ... 
380    334
381    335
382    340
383    322
384    343
Name: adjusted_gre, Length: 385, dtype: int64

In [47]:
admissions.head()

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.193197,1,347
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,1,326
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,1,332
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,1,324
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,340


In [48]:
bins = pd.cut(admissions['adjusted_gre'], 4)

In [49]:
bins.value_counts()

(325.0, 337.5]     140
(312.5, 325.0]     120
(337.5, 350.0]      70
(299.95, 312.5]     55
Name: adjusted_gre, dtype: int64